In [33]:
# ============================================================================
#                               IMPORTING LIBRARIES
# ============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

from tqdm import tqdm
import time
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold,KFold, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
import xgboost as xgb
from sklearn import preprocessing, model_selection, pipeline
from sklearn.metrics import f1_score, roc_auc_score

from tensorflow_core.python.keras.models import Sequential
from tensorflow_core.python.keras.layers.recurrent import LSTM, GRU
from tensorflow_core.python.keras.layers.core import Dense, Activation, Dropout
from tensorflow_core.python.keras.layers.embeddings import Embedding
from tensorflow_core.python.keras.layers.normalization import BatchNormalization
from tensorflow_core.python.keras.utils import np_utils
from tensorflow_core.python.keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from tensorflow_core.python.keras.preprocessing import sequence, text
from tensorflow_core.python.keras.callbacks import EarlyStopping

import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

SymbolAlreadyExposedError: Symbol pad_sequences is already exposed as ('keras.preprocessing.sequence.pad_sequences',).

In [ ]:
# ============================================================================
#                               FEATURE EXTRACTION
# ============================================================================
# ----------------------
# CREATING INITIAL DF
# ----------------------
df = pd.read_csv("./input/fake_job_postings.csv")
df.isnull().sum()

In [ ]:
# --------------------------
# EXTRACTING TEXT FEATURES
# --------------------------
text_df = df[['title', 'company_profile', 'description', 'requirements',
              'benefits', 'fraudulent']]
text_df = text_df.fillna(' ')
text_df.head()

In [ ]:
# ------------------------------------
# EXTRACTING CATEGORICAL FEATURES
# ------------------------------------
cat_df = df[['telecommuting', 'has_company_logo', 'has_questions',
             'employment_type', 'required_experience', 'required_education',
             'industry', 'function', 'fraudulent']]
cat_df = cat_df.fillna('None')
cat_df.head()

In [ ]:
# -----------------------
# INITIALIZING TARGET
# -----------------------
fig, axes = plt.subplots(ncols=2, figsize=(17, 5), dpi=(100))
plt.tight_layout()

df['fraudulent'].value_counts().plot(kind='pie', ax=axes[0],
                                     labels=['Real Post (95%)', 'Fake Post (5%)'])
temp = df["fraudulent"].value_counts()
sns.barplot(temp.index, temp, ax=axes[1])

axes[0].set_ylabel(' ')
axes[1].set_ylabel(' ')
axes[1].set_xticklabels(['Real Post (17014)', 'Fake Post (866)'])

axes[0].set_title('Target Distribution in Dataset', fontsize=13)
axes[1].set_title('Target Count in Dataset', fontsize=13)

plt.show()

In [ ]:
# ============================================================================
#                      EXPLORATORY ANALYSIS OF JOB POSTINGS
# ============================================================================
cat_cols = ['telecommuting', 'has_company_logo', 'has_questions',
            'employment_type', 'required_experience', 'required_education']
import matplotlib.gridspec as gridspec
grid = gridspec.GridSpec(3, 3, wspace=0.5, hspace=0.5)
plt.figure(figsize=(15, 25))

for n, col in enumerate(cat_df[cat_cols]):
    ax = plt.subplot(grid[n]) # feeding the figure of grid
    sns.countplot(x=col, data=cat_df, hue='fraudulent', palette='Set2')
    ax.set_ylabel('Count', fontsize=12) # y axis label
    ax.set_title(f'{col} Distribution by Target', fontsize=15) # title label
    ax.set_xlabel(f'{col} values', fontsize=12) # x axis label
    xlabels = ax.get_xticklabels()
    ylabels = ax.get_yticklabels()
    ax.set_xticklabels(xlabels,  fontsize=10)
    ax.set_yticklabels(ylabels,  fontsize=10)
    plt.legend(fontsize=8)
    plt.xticks(rotation=90)
    total = len(cat_df)
    sizes=[] # Get highest values in y
    for p in ax.patches: # loop to all objects
        height = p.get_height()
        sizes.append(height)
        ax.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(height/total*100),
                ha="center", fontsize=10)
    ax.set_ylim(0, max(sizes) * 1.15)

plt.show()

In [ ]:
# ------------------------------------------------------------
# EVALUATING NUMBER OF CHARACTERS IN COMPANY PROFILE
# ------------------------------------------------------------
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(17, 5), dpi=100)
length=text_df[text_df['fraudulent']==1]['description'].str.len()
ax1.hist(length, bins=20, color='orangered')
ax1.set_title('Fake Posts')
length=text_df[text_df['fraudulent']==0]['description'].str.len()
ax2.hist(length, bins=20)
ax2.set_title('Real Posts')
fig.suptitle('Characters in Company Description')
plt.show()

In [ ]:
# --------------------------------------------------------
# EVALUATING NUMBER OF CHARACTERS IN JOB REQUIREMENTS
# --------------------------------------------------------
fig,(ax1,ax2)= plt.subplots(ncols=2, figsize=(17, 5), dpi=100)
length=text_df[text_df["fraudulent"]==1]['requirements'].str.len()
ax1.hist(length,bins = 20,color='orangered')
ax1.set_title('Fake Posts')
length=text_df[text_df["fraudulent"]==0]['requirements'].str.len()
ax2.hist(length,bins = 20)
ax2.set_title('Real Posts')
fig.suptitle('Characters in requirements')
plt.show()

In [ ]:
# --------------------------------------------------------
# EVALUATING NUMBER OF CHARACTERS IN JOB BENEFITS
# --------------------------------------------------------
fig,(ax1,ax2)= plt.subplots(ncols=2, figsize=(17, 5), dpi=100)
length=text_df[text_df["fraudulent"]==1]['benefits'].str.len()
ax1.hist(length,bins = 20,color='orangered')
ax1.set_title('Fake Post')
length=text_df[text_df["fraudulent"]==0]['benefits'].str.len()
ax2.hist(length,bins = 20)
ax2.set_title('Real Post')
fig.suptitle('Characters in benefits')
plt.show()

In [ ]:
# ============================================================================
#                               DATA PREPROCESSING
# ============================================================================
text = text_df[text_df.columns[0:-1]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
target = df['fraudulent']

print(len(text))
print(len(target))

In [ ]:
# --------------------------------------------------------
# ANALYSIS OF UNIGRAMS AND BIGRAMS
# --------------------------------------------------------
def get_top_tweet_unigrams(corpus, n=None):
    vec = CountVectorizer(ngram_range=(1, 1)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

def get_top_tweet_bigrams(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

fig, axes = plt.subplots(ncols=2, figsize=(18, 30), dpi=100)
plt.tight_layout()

top_unigrams=get_top_tweet_unigrams(text)[:50]
x,y=map(list,zip(*top_unigrams))
sns.barplot(x=y,y=x, ax=axes[0], color='teal')


top_bigrams=get_top_tweet_bigrams(text)[:50]
x,y=map(list,zip(*top_bigrams))
sns.barplot(x=y,y=x, ax=axes[1], color='crimson')


axes[0].set_ylabel(' ')
axes[1].set_ylabel(' ')

axes[0].set_title('Top 50 most common unigrams in text', fontsize=15)
axes[1].set_title('Top 50 most common bigrams in text', fontsize=15)

plt.show()

In [ ]:
# ---------------------------
# TEXT CLEANING
# ---------------------------
def clean_text(text):
    '''MAKE TEXT LOWERCASE, REMOVE TEXT IN SQUARE BRACKETS,REMOVE LINKS,REMOVE PUNCTUATION
    AND REMOVE WORDS CONTAINING NUMBERS.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

text = text.apply(lambda x: clean_text(x))
text.head()

In [ ]:
# --------------------------
# CREATING TOKENIZER
# --------------------------

tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

# APPLYING TOKENIZER
text = text.apply(lambda x: tokenizer.tokenize(x))
stop_words = stopwords.words('english')

def remove_stopwords(text):
    words = [w for w in text if w not in stop_words]
    return words
text = text.apply(lambda x : remove_stopwords(x))

def combine_text(list_of_text):
    combined_text = ' '.join(list_of_text)
    return combined_text
text = text.apply(lambda x: combine_text(x))
text.head()

In [ ]:
# ============================================================================
#                             BASELINE MODEL
# ============================================================================
# ------------------
# INITIALIZE MODEL
# ------------------
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
auc_buf = []
count = 0
predictions = 0

# ------------------------------------------------------
# ENUMERATES THE SPLITS AND SUMMARIZES THE PREDICTIONS
# ------------------------------------------------------
for train_ix, test_ix in kfold.split(text, target):
    print('Fold {}'.format(count + 1))
    train_X, test_X = text[train_ix], text[test_ix]
    train_y, test_y = target[train_ix], target[test_ix]

    # Appling Count Vectorizer
    count_vectorizer = CountVectorizer()
    train_X_vec = count_vectorizer.fit_transform(train_X)
    test_X_vec = count_vectorizer.transform(test_X)

    lr = LogisticRegression(C=0.1, solver='lbfgs', max_iter=1000, verbose=0, n_jobs=-1)
    lr.fit(train_X_vec, train_y)
    preds = lr.predict(test_X_vec)

    auc = roc_auc_score(test_y, preds)
    print('{} AUC: {}'.format(count, auc))
    auc_buf.append(auc)
    count += 1

print('AUC mean score = {:.6f}'.format(np.mean(auc_buf)))
print('AUC std score = {:.6f}'.format(np.std(auc_buf)))

In [ ]:
# -------------------------------------
# SPLITTING THE DATA FOR THE GLOVE
# -------------------------------------
X_train, X_test, y_train, y_test = \
    train_test_split(text, target, test_size=0.2, random_state=4, stratify=target)

In [ ]:
# --------------------------------------------
# LOADS THE GLOVE VECTORS INTO A DICTIONARY
# --------------------------------------------
embeddings_index={}
with open('../input/glove-global-vectors-for-word-representation/glove.6B.200d.txt','r') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embeddings_index[word]=vectors
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
# ----------------------------------------------------
# CREATES A NORMALIZED VECTOR FOR THE WHOLE SENTENCE
# ----------------------------------------------------
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(200)
    return v / np.sqrt((v ** 2).sum())

In [ ]:
# --------------------------
# CREATES GLOVE FEATURES
# --------------------------
xtrain_glove = np.array([sent2vec(x) for x in tqdm(X_train)])
xtest_glove = np.array([sent2vec(x) for x in tqdm(X_test)])

In [ ]:
# ============================================================================
#                          DEEP LEARNING TIME BABY
# ============================================================================

# ----------------------------------------------
# SCALING THE DATA BEFORE CREATING NEURAL NET
# ----------------------------------------------
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xtest_glove_scl = scl.transform(xtest_glove)

In [ ]:
# -------------------------------------------
# CREATING A 2 LAYER SEQUENTIAL NEURAL NET
# -------------------------------------------
model = Sequential()

model.add(Dense(200, input_dim=200, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(1))
model.add(Activation('sigmoid'))

# COMPILE THE MODEL
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# ---------------
# FITTING MODEL
# ---------------
model.fit(xtrain_glove_scl, y=y_train, batch_size=64,
          epochs=10, verbose=1,
          validation_data=(xtest_glove_scl, y_test))

In [ ]:
# -----------------------
# MAKING PREDICTIONS
# -----------------------
predictions = model.predict(xtest_glove_scl)
predictions = np.round(predictions).astype(int)
print('2 layer sequential neural net on GloVe Feature')
print ("AUC score :", np.round(roc_auc_score(y_test, predictions),5))

In [ ]:
# -----------------------------
# SIMPLE BERT IMPLEMENTATION
# -----------------------------
new_text = text_df[text_df.columns[0:-1]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
target = df['fraudulent']

def clean_text(text):
    '''MAKE TEXT LOWERCASE, REMOVE TEXT IN SQUARE BRACKETS,REMOVE LINKS,REMOVE PUNCTUATION
    AND REMOVE WORDS CONTAINING NUMBERS.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text


# APPLYING THE CLEANING FUNCTION TO BOTH TEST AND TRAINING DATASETS
new_text = new_text.apply(lambda x: clean_text(x))
new_text.head(3)

# TRYING FIRST 2000 SAMPLE
batch_1 = new_text[:2000]
target_1 = target[:2000]
target_1.value_counts()

# DISTILBERT
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# LOAD PRETRAINED MODEL/TOKENIZER
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
# ----------------
# TOKENIZATION
# ----------------
tokenized = batch_1.apply((lambda x: tokenizer.encode(x, max_length = 60, add_special_tokens=True)))
tokenized.shape

In [ ]:
# -------------------------------------------
# PADDING ==> CONVERT 1D ARRAY TO 2D ARRAY
# -------------------------------------------
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
np.array(padded).shape

In [ ]:
# -----------------------------------------------------
# MASKING ==>  IGNORE (MASK) THE PADDING WE'VE ADDED
# -----------------------------------------------------
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

In [ ]:
# ----------------
# DEEP LEARNING
# ----------------
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

last_hidden_states[0].shape

In [ ]:
# ---------------
# MODELING
# ---------------
features = last_hidden_states[0][:,0,:].numpy()
labels = target_1

train_features, test_features, train_labels, test_labels = \
    train_test_split(features, labels)

lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

predictions = lr_clf.predict(test_features)
predictions = np.round(predictions).astype(int)
print ("AUC score :", np.round(roc_auc_score(test_labels, predictions),5))